In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC

target = 'moved_after_2019'
idx = 'user_id'
root = '/Users/senaemirler/garanti-bbva-data-camp/'


In [2]:
df_train = pd.read_csv(os.path.join(root, 'train_users.csv'))
df_test  = pd.read_csv(os.path.join(root, 'test_users.csv'))
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_edu   = pd.read_csv(os.path.join(root, 'education.csv'))
df_skill = pd.read_csv(os.path.join(root, 'skills.csv'))
df_exp   = pd.read_csv(os.path.join(root, 'work_experiences.csv'))

In [3]:
df_edu

,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month
0,0,Anadolu Üniversitesi,NaN,NaN,NaN,NaN
1,0,Adıyaman Üniversitesi,NaN,NaN,NaN,NaN
2,0,Fırat Üniversitesi,NaN,NaN,NaN,NaN
3,1,Fırat Üniversitesi,Yüksek Lisans,Yazılım Mühendisliği,NaN,NaN
4,1,Fırat Üniversitesi,Lisans,Yazılım Mühendisliği,NaN,NaN
...,...,...,...,...,...,...
142570,66272,Halmstad University,NaN,Computer Science,NaN,NaN
142571,66272,Bogazici University,NaN,Chemistry,NaN,NaN
142572,66273,Istanbul Technical University,Bachelor's degree,Electronic and Communication Engineering,NaN,NaN
142573,66273,Bogazici University,Master's degree,Electrical and Electronics Engineering,201509.0,NaN


In [4]:
df_edu.fillna({'school_name':'other'}, inplace=True)

In [5]:
df_edu = df_edu.applymap(lambda x: x.lower() if type(x) == str else x)


In [29]:
import re

df_edu['school_name'] = df_edu['school_name'].apply(lambda x: 'other' if re.search('i̇.ö.o|orta okul|i̇smek|camp|kamp|okulları|bilge adam |koleji|koleji|udacity|i̇lköğretim|bilge adam|kolej|ilk ögretim|ilkogretim|i̇lkokul|lse|ilköğretim|ilkolul|lise|high school', str(x).lower()) else x)




In [7]:
counts = df_edu['school_name'].value_counts()
df_edu['school_name'] = df_edu['school_name'].apply(lambda x: 'other' if counts[x] == 1 else x)


In [26]:
df_edu.loc[df_edu['school_name'].str.contains("kocaeli|gebze", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "kocaeli üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("fırat|firat", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "fırat üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("i̇stanbul technical|i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu|istanbul technical university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("marmara", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "marmara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("anadolu university|anadolu üniversitesi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "anadolu üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("sakarya", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "sakarya üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("yıldız teknik|ytü|ytu|yildiz|yıldız", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "yıldız teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("hacettepe", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "hacettepe üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ege", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ege üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("istanbul üniversitesi|istanbul university|i̇stanbul üniversitesi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("osmangazi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "osmangazi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("gazi|gazı", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "gazi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("karadeniz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "karadeniz teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("eylül|eylul", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "dokuz eylül üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("bahcesehir|bahçeşehir", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "bahçeşehir üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("selçuk", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "selçuk üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("beykent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "beykent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ankara üniversitesi|ankara universitesi|ankara univercity|ankara university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ankara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("trakya", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "trakya üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("süleyman|suleyman", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "süleyman demirel üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("bosphorus|boğaziçi university|boğaziçi üniversitesi|boun|boün|bogaz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "boğaziçi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("bilkent üniversitesi|bilkent university|bilkent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "bilkent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ankara üniversitesi|ankara universitesi|ankara univercity|ankara university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ankara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("erciyes", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "erciyes üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("çukurova|cukurova", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "çukurova üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("tobb", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "tobb üniversitesi"


df_edu.loc[df_edu['school_name'].str.contains("metu|odtü|middle east|orta doğu| orta dogu", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "orta doğu teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("yeditepe", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "yeditepe üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("sabanci|sabancı", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "sabancı üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("mugla|muğla", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "muğla üniversitesi"


df_edu.loc[df_edu['school_name'].str.contains("koç|koc", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "koç üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ahmet yesevi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ahmet yesevi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("istanbul bilgi|i̇stanbul bilgi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul bilgi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("atatürk|ataturk", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atatürk üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("eskişehir|eskisehir", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "eskişehir üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("başkent|baskent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "başkent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("galatasaray", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "galatasaray üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("atılım|atilim", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atılım üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("celal", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "celal bayar üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("galatasaray", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "galatasaray üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("uludağ|uludag", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "uludağ üniversitesi"







In [22]:
school_name_conv = {
    "(?i).*(otomotiv).*": "otomotiv sektörü",
    "(?i).*(duzce|düzce|düzce).*": "düzce üniversitesi",
    "(?i).*(pamukkale).*": "pamukkale üniversitesi",
    "(?i).*(çankaya|cankaya).*": "çankaya üniversitesi",
    "(?i).*(maltepe).*": "maltepe üniversitesi",
    "(?i).*(karabuk|karabük).*": "karabük üniversitesi",
    "(?i).*(selcuk|selçuk).*": "pamukkale üniversitesi",
    "(?i).*(okan).*": "okan üniversitesi",
    "(?i).*(özyeğin|ozyegin|özyeğin university).*": "özyeğin üniversitesi",
    "(?i).*(i̇stanbul aydın|istanbul aydın|i̇stanbul aydin).*": "istanbul aydın üniversitesi",
    "(?i).*(zaim).*": "zaim üniversitesi",
    "(?i).*(katip çelebi|katip celebi).*": "katip çelebi üniversitesi",
    "(?i).*(mimar sinan).*": "mimar sinan üniversitesi",
    "(?i).*(yaşar|yasar).*": "yaşar üniversitesi",
    "(?i).*(çanakkale|canakkale|onsekiz mart).*": "çanakkale onsekiz mart üniversitesi",
    "(?i).*(selcuk|selçuk).*": "selçuk üniversitesi",
    "(?i).*(balıkesir|balikesir).*": "balıkesir üniversitesi",
    "(?i).*(i̇zmir ekonomi|izmir ekonomi).*": "izmir ekonomi üniversitesi",
    "(?i).*(haliç|halic).*": "haliç üniversitesi",
    "(?i).*(fatih).*": "fatih üniversitesi",
    "(?i).*(abant).*": "abant üniversitesi",
    "(?i).*(cerrahpaşa).*": "istanbul üniversitesi",
    "(?i).*(doğu akdeniz|dogu akdeniz|eastern mediterranean ).*": "doğu akdeniz üniversitesi",
    "(?i).*(i̇nönü|inönü|ınonu).*": "akdeniz üniversitesi",
    "(?i).*(dumlup).*": "dumlupınar üniversitesi",
    "(?i).*(namık kema|namik kema).*": "namık kema üniversitesi",
    "(?i).*(i̇stanbul gelişim|istanbul gelişim|istanbul gelisim).*": "istanbul gelişim üniversitesi",
    "(?i).*(istanbul kultur|istanbul kültür).*": "istanbul kultur üniversitesi",
    "(?i).*(bilecik).*": "bilecik üniversitesi",
    "(?i).*(üsküdar|uskudar).*": "üsküdar üniversitesi",
    "(?i).*(iskenderun|i̇skenderun).*": "iskenderun üniversitesi",
    "(?i).*(stanford).*": "stanford üniversitesi",
    "(?i).*(karatay).*": "karatay üniversitesi",
    "(?i).*(kadir has).*": "kadir has üniversitesi",
    "(?i).*(sivas).*": "sivas üniversitesi",
    "(?i).*(işık|ışık).*": "ışık üniversitesi",
    
}

df_edu.loc[:, 'school_name'] = df_edu.loc[:, 'school_name'].replace(school_name_conv, regex=True)


In [30]:
value_counts = df_edu['school_name'].value_counts().sort_values(ascending=False)
num_unique = df_edu['school_name'].nunique()
print(f"There are {num_unique} unique values in the 'school_name' column.")

unique_values_degree = df_edu['school_name'].unique()
for value, count in value_counts.items():
    print(f"{value}: {count}")

There are 132 unique values in the 'school_name' column.
other: 30080
istanbul teknik üniversitesi: 9875
anadolu üniversitesi: 8034
koç üniversitesi: 7673
istanbul üniversitesi: 7492
yıldız teknik üniversitesi: 6754
gazi üniversitesi: 5928
sakarya üniversitesi: 5344
marmara üniversitesi: 5047
ege üniversitesi: 4469
hacettepe üniversitesi: 4133
bahçeşehir üniversitesi: 3615
dokuz eylül üniversitesi: 3136
boğaziçi üniversitesi: 3101
bilkent üniversitesi: 2865
ankara üniversitesi: 2356
beykent üniversitesi: 2209
selçuk üniversitesi: 2164
trakya üniversitesi: 2126
süleyman demirel üniversitesi: 2042
karadeniz teknik üniversitesi: 2020
fırat üniversitesi: 1733
orta doğu teknik üniversitesi: 1546
erciyes üniversitesi: 1511
yeditepe üniversitesi: 1484
çukurova üniversitesi: 1432
sabancı üniversitesi: 1278
istanbul bilgi üniversitesi: 1146
tobb üniversitesi: 1142
akdeniz üniversitesi: 900
muğla üniversitesi: 637
ahmet yesevi üniversitesi: 513
atatürk üniversitesi: 462
eskişehir üniversitesi: 4

In [15]:
# 'fields_of_study' sütunundaki değerlerin sayısını sayma
counts = df_edu['school_name'].value_counts()

# 20'den az tekrarlanan değerleri seçme
rare_values = counts[counts < 27].index.tolist()

# 'other' olarak değiştirme
df_edu.loc[df_edu['school_name'].isin(rare_values), 'school_name'] = 'other'

In [ ]:
unique_values_location = df_edu['school_name'].unique()

location_list = []
for value in unique_values_location:
    if 'ilk' in value.lower():
        location = value.strip().title()
        location_list.append(location)

print(location_list)


In [ ]:
degree_counts = df_edu['school_name'].value_counts().head(30)
print(degree_counts)